<h1 align="center"><a href="https://github.com/sborquez/her2bdl"> Her2BDL</a> - Her2 Bayesian Deep Learning</h1>


<br>
<center>
<img src="images/utfsm.png" width="50%", align="center"/>
</center>
<h2 align="center">Model Inspection</h2>

<center>
<i> Notebook created by Sebastián Bórquez G. - <a href="mailto://sebstian.borquez.g@gmail.com">sebastian.borquez.g@gmail.com</a> - utfsm - August 2020.</i>
</center>


# Setup Notebook

The first step is to sync this notebook with Google Drive, and change directory to gerumo repository.

In [1]:
#!export HER2BDL_HOME="/home/sborquez/projects/her2bdl"
#!export HER2BDL_DATASETS="/home/sborquez/projects/her2bdl/datasets"
#!export HER2BDL_EXPERIMENTS="/home/sborquez/projects/her2bdl/experiments"
#!export HER2BDL_EXTRAS="/home/sborquez/projects/her2bdl-files"

In [2]:
## (Option A) Colab Setup
# Connect to your `Google Drive` and install some dependencies.
try:
    raise NotImplementedError
    from google.colab import drive
    colab_ = True
    drive.mount('/content/drive')
    #!pip install git+https://github.com/sborquez/her2bdl\
## (Option B) Local Setup
# Run locally.
except:
    colab_ = False
    
# Adhoc modules
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_DISABLE_PTX_JIT'] = "1"
os.environ["WANDB_MODE"] = 'dryrun'

from extra import *

# Her2BDL package
#if is not installed run this line
HER2BDL_HOME=os.environ.get("HER2BDL_HOME")
%cd $HER2BDL_HOME

from her2bdl import *
%matplotlib inline

/home/asuka/projects/her2bdl


In [3]:
config_folder = Path(HER2BDL_HOME) / "scripts" / "config"

# Model Summary

In [4]:
# Binary
#config_file = config_folder / "binary_classification/efficientnet_b0_binary_classifier.yaml"
#config_file = config_folder / "aleatoric/efficientnet_b0_binary_classifier.yaml"
# HER2
config_file = config_folder / "baseline/hdxconv_c876_d02.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d05.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d08.yaml"
#config_file = config_folder / "aleatoric/hdxconv_c876.yaml"

In [5]:
config = load_config_file(config_file)
run_dir = setup_experiment(config, mode="debug")

# Experiment paths and indentifiers
experiments_folder = config["experiment"]["experiments_folder"]
experiment_name    = config["experiment"]["name"]
experiment_folder  = Path(experiments_folder) / experiment_name
run_id             = config["experiment"]["run_id"]

# Training parameters
epochs = config["training"]["epochs"]
batch_size  = config["training"]["batch_size"]
# Model architecture
model_configuration = config["model"]
# Dataset
data_configuration = config["data"]
generators, input_shape, num_classes, labels = setup_generators(
    batch_size=batch_size, **data_configuration
)

wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
/home/asuka/projects/her2bdl/her2bdl/data/dataset.py:224: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[col] = dataset[col].str.replace(*mapper)
100%|██████████| 34/34 [00:33<00:00,  1.01it/s]
/home/asuka/projects/her2bdl/her2bdl/data/dataset.py:224: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset[col] = dataset[col].str.replace(*mapper)
100%|██████████| 11/11 [00:00<00:00, 21.97it/s]


# Model Summary

Load and summary models.

In [6]:
# Epistemic
epistemic_model = setup_model(input_shape, num_classes, **model_configuration, 
                              build=True, build_aleatoric=True)
aleatoric_model = epistemic_model.get_aleatoric_model()


## Epistemic Model

In [7]:
epistemic_model.summary()

Model: "hed_conv_classifier_mc_dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 10816)             303352    
_________________________________________________________________
classifier (Functional)      (None, 4)                 2812356   
_________________________________________________________________
aleatoric_model (AleatoricMo multiple                  3115773   
Total params: 6,231,481
Trainable params: 6,227,801
Non-trainable params: 3,680
_________________________________________________________________


### Encoder

In [8]:
encoder = epistemic_model.layers[0]
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
stain_separator (Separate_HE (None, 240, 240, 3)       0         
_________________________________________________________________
scaler (Lambda)              (None, 240, 240, 3)       0         
_________________________________________________________________
stain_depthwiseConv2D (Depth (None, 118, 118, 24)      600       
_________________________________________________________________
depthwiseConv2D_batchnorm (B (None, 118, 118, 24)      96        
_________________________________________________________________
depthwiseConv2D_activation ( (None, 118, 118, 24)      0         
_________________________________________________________________
block0_conv2d_a (Conv2D)     (None, 118, 118, 32)      6912

### Classifier 

In [9]:
classifier = epistemic_model.layers[1]
classifier.summary()

Model: "classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 10816)]           0         
_________________________________________________________________
head_dense_1 (Dense)         (None, 256)               2769152   
_________________________________________________________________
head_batchnorm_1 (BatchNorma (None, 256)               1024      
_________________________________________________________________
head_activation_1 (Activatio (None, 256)               0         
_________________________________________________________________
head_mc_dropout_1 (Dropout)  (None, 256)               0         
_________________________________________________________________
head_dense_2 (Dense)         (None, 128)               32896     
_________________________________________________________________
head_batchnorm_2 (BatchNorma (None, 128)               5

## Aleatoric Model

In [10]:
aleatoric_model.summary()

Model: "aleatoric_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 10816)             303352    
_________________________________________________________________
aleatoric_classifier (Functi (None, 5)                 2812421   
Total params: 3,115,773
Trainable params: 3,113,933
Non-trainable params: 1,840
_________________________________________________________________


### Encoder

In [11]:
encoder = aleatoric_model.layers[0]
encoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240, 240, 3)]     0         
_________________________________________________________________
stain_separator (Separate_HE (None, 240, 240, 3)       0         
_________________________________________________________________
scaler (Lambda)              (None, 240, 240, 3)       0         
_________________________________________________________________
stain_depthwiseConv2D (Depth (None, 118, 118, 24)      600       
_________________________________________________________________
depthwiseConv2D_batchnorm (B (None, 118, 118, 24)      96        
_________________________________________________________________
depthwiseConv2D_activation ( (None, 118, 118, 24)      0         
_________________________________________________________________
block0_conv2d_a (Conv2D)     (None, 118, 118, 32)      6912

### Aleatoric Classifier

In [12]:
aleatoric_classifier  = aleatoric_model.layers[1]
aleatoric_classifier.summary()

Model: "aleatoric_classifier"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10816)]      0                                            
__________________________________________________________________________________________________
head_dense_1 (Dense)            (None, 256)          2769152     input_2[0][0]                    
__________________________________________________________________________________________________
head_batchnorm_1 (BatchNormaliz (None, 256)          1024        head_dense_1[0][0]               
__________________________________________________________________________________________________
head_activation_1 (Activation)  (None, 256)          0           head_batchnorm_1[0][0]           
_______________________________________________________________________________

# Forward Pass Times

In [5]:
from extra import profiling

In [6]:
# EfficientNet
# Load configuration and model
config_file = config_folder / "binary_classification/efficientnet_b0_binary_classifier.yaml"
config = load_config_file(config_file)
## build model
num_classes = config["data"]["num_classes"]
input_shape = (config["data"]["img_width"],config["data"]["img_height"], config["data"]["img_channels"])
model = setup_model(input_shape, num_classes, **(config["model"]), build=True)    
profiling(config, model)

Model: "efficient_net_mc_dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 62720)             4049571   
_________________________________________________________________
classifier (Functional)      (None, 2)                 16099650  
Total params: 20,149,221
Trainable params: 16,098,754
Non-trainable params: 4,050,467
_________________________________________________________________
input shape (224, 224, 3)
batch size: 16
Sample size: 200
MC dropout batch size: 32
Classic Fordward Pass
Model
	mean: 4.404709585200180299580097198486 [s]
	std err: 13.530030795150183919872688420583 [s]
Encoder
	mean: 0.191823579599440574572355444616 [s]
	std err: 0.202334637659077665183815497585 [s]
Classifier
	mean: 0.061652878198947290433462598003 [s]
	std err: 0.015856618853730894413978091961 [s]

Vanilla Predictive Distribution
Model
	mean: 4.066107045200624270364642143250 

In [7]:
# HER2
# Load configuration and model
config_file = config_folder / "baseline/hdxconv_c876_d02.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d05.yaml"
#config_file = config_folder / "baseline/hdxconv_c876_d08.yaml"
config = load_config_file(config_file)
## build model
num_classes = config["data"]["num_classes"]
input_shape = (config["data"]["img_width"],config["data"]["img_height"], config["data"]["img_channels"])
model = setup_model(input_shape, num_classes, **(config["model"]), build=True)    
profiling(config, model)

Model: "hed_conv_classifier_mc_dropout"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder (Functional)         (None, 10816)             303352    
_________________________________________________________________
classifier (Functional)      (None, 4)                 2812356   
Total params: 3,115,708
Trainable params: 3,113,868
Non-trainable params: 1,840
_________________________________________________________________
input shape (240, 240, 3)
batch size: 64
Sample size: 200
MC dropout batch size: 64
Classic Fordward Pass
Model
	mean: 1.733722059900173917412757873535 [s]
	std err: 4.790924828581483829736953339307 [s]
Encoder
	mean: 0.228370267699938261918291004804 [s]
	std err: 0.025977574770906359385991279964 [s]
Classifier
	mean: 0.059881717499956722827736399495 [s]
	std err: 0.017108786379268905303296932630 [s]

Vanilla Predictive Distribution
Model
	mean: 13.953310893700109573956069652922 